In [1]:
#in my Windows the optimizers are located in
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/tensorflow/python/keras
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\tensorflow\python\keras
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/sklearn/neural_network
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\sklearn\neural_network
#rememmber to start the notebook using the environment tensorflow


from numpy.random import seed
import numpy as np
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import tensorflow as tf
# from tensorflow.python.keras.optimizers import Fire
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers

from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import matplotlib.pyplot as plt
%matplotlib inline

### CrossEntropyLoss
The input comes from a softmax

        output /= tf.reduce_sum(output,
                                reduction_indices=len(output.get_shape()) - 1,
                                keep_dims=True)
        # manual computation of crossentropy
        epsilon = _to_tensor(_EPSILON, output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon, 1. - epsilon)
        return - tf.reduce_sum(target * tf.log(output),
                               reduction_indices=len(output.get_shape()) - 1)


### Softmax
softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), axis)


## Preparing the input data


In [19]:
#Preparing the input data

#loading the MNIST dataset
mnist = tf.keras.datasets.mnist

#Separating into train and test (60000 train, 10000 test)
(x_train, y_train0),(x_test, y_test0) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#flattening the images (from 28x28 to 784)
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

#converting the otputs (labels) into one hot vectors
lb = preprocessing.LabelBinarizer()
lb.fit(np.array([0,1,2,3,4,5,6,7,8,9]))
y_train = lb.transform(y_train0)
y_test = lb.transform(y_test0)

## Defining the input parameters


In [20]:
#Defining the input parameters
BatchSize         = 6000
NeuronsLayer1     = 100
NeuronsLayer2     = 100
Epochs            = 100

## Model in tensorflow/Keras


In [21]:
# Defining the net

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(NeuronsLayer1,
                        activation=tf.nn.relu),
  tf.keras.layers.Dense(NeuronsLayer2, 
                        activation=tf.nn.relu),
  #  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, 
                        activation=tf.nn.softmax)
                                   ])
print(model)

In [22]:
# Construct our model, # Construct our loss function and an Optimizer. 

loss = tf.keras.backend.categorical_crossentropy
optimizer = optimizers.Adam()

model.compile(optimizer= optimizer,
              loss=loss ) #'categorical_crossentropy')

In [23]:
#train the network

HistoryKeras = model.fit(  x_train, y_train, 
            epochs=Epochs,
            batch_size=BatchSize)


Epoch 1/100
60000/60000 [==============================] - 2s 27us/step - loss: 2.0118
Epoch 2/100
60000/60000 [==============================] - 2s 26us/step - loss: 1.1902
Epoch 3/100
60000/60000 [==============================] - 1s 24us/step - loss: 0.6249
Epoch 4/100
60000/60000 [==============================] - 1s 23us/step - loss: 0.4266
Epoch 5/100
60000/60000 [==============================] - 1s 23us/step - loss: 0.3521
Epoch 6/100
60000/60000 [==============================] - 2s 25us/step - loss: 0.3120
Epoch 7/100
60000/60000 [==============================] - 1s 22us/step - loss: 0.2845
Epoch 8/100
60000/60000 [==============================] - 1s 22us/step - loss: 0.2636
Epoch 9/100
60000/60000 [==============================] - 2s 28us/step - loss: 0.2457
Epoch 10/100
60000/60000 [==============================] - 1s 24us/step - loss: 0.2303
Epoch 11/100
60000/60000 [==============================] - 1s 24us/step - loss: 0.2165
Epoch 12/100
60000/60000 [===============

In [24]:
#Predicting

#model.evaluate(x_test, y_test)
y_train_hat_keras = model.predict_proba(x_train)

log_loss(y_train, y_train_hat_keras)

0.013767337911313926

In [25]:
print(y_train[0])
print(y_train[1])

[0 0 0 0 0 1 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]


In [9]:
print(y_train_hat_keras[0])
print(y_train_hat_keras[1])

[3.9880906e-14 6.0275569e-13 1.0921624e-11 7.2521344e-03 5.0610332e-22
 9.9274784e-01 1.1621728e-15 1.3641267e-11 5.8727120e-12 8.0995748e-12]
[9.9999821e-01 2.7508361e-13 1.8465716e-06 3.5425711e-11 1.1048017e-13
 1.4060317e-14 2.6582250e-09 2.1069781e-11 3.7640281e-14 5.1660735e-11]


In [10]:
#Show aprameters of the network

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  78500     
_________________________________________________________________
dense_1 (Dense)              multiple                  10100     
_________________________________________________________________
dense_2 (Dense)              multiple                  1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________
